In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk-15.0.1"

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
 cd Mecab-ko-for-Google-Colab

In [ ]:
! bash install_mecab-ko_on_colab190912.sh

세션 재실행 필요

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
import os 
os.chdir('/content/drive/MyDrive/dacon')

In [ ]:
import re
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('news_train.csv')
test = pd.read_csv('news_test.csv')
submission = pd.read_csv('sample_submission.csv')

In [ ]:
info_counts = train['info'].value_counts()
sns.barplot(info_counts.index, info_counts)
plt.gca().set_ylabel('info')

In [ ]:
def text_preprocessing(text_list):
    
    stopwords = ['을', '를', '이', '가', '은', '는', 'null'] #불용어 설정
    tokenizer = Mecab() #형태소 분석기 
    token_list = []
    
    for text in text_list:
        txt = re.sub('[^가-힣a-z]', ' ', text) #한글과 영어 소문자만 남기고 다른 글자 모두 제거
        token = tokenizer.morphs(txt) #형태소 분석
        token = [t for t in token if t not in stopwords or type(t) != float] #형태소 분석 결과 중 stopwords에 해당하지 않는 것만 추출
        token_list.append(token)
        
    return token_list, tokenizer

train['new_article'], mecab = text_preprocessing(train['content']) 

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def text2sequence(train_text, max_len=100):
    
    tokenizer = Tokenizer() #keras의 vectorizing 함수 호출
    tokenizer.fit_on_texts(train_text) #train 문장에 fit
    train_X_seq = tokenizer.texts_to_sequences(train_text) #각 토큰들에 정수 부여
    vocab_size = len(tokenizer.word_index) + 1 #모델에 알려줄 vocabulary의 크기 계산
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len) #설정한 문장의 최대 길이만큼 padding
    
    return X_train, vocab_size, tokenizer

train_X, vocab_size, vectorizer = text2sequence(train['content'], max_len = 100)

In [ ]:
glove = dict()
f = open('./glove.txt')
for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    glove[word] = vector
f.close()

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))

for idx, word in enumerate(list(vectorizer.word_index.keys())):  #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in glove: #넘겨 받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = glove[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_matrix[idx] = embedding_vector #해당 위치의 embedding_mxtrix에 저장합니다.
    else:
        print("glove 없는 단어입니다.")
        break

In [ ]:
import tensorflow as tf

def LSTM(vocab_size, max_len=1000):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(vocab_size, 300,weights = [embedding_matrx], input_length = max_len)) #임베딩 가중치 적용 코드
    model.add(tf.keras.layers.SpatialDropout1D(0.3))
    model.add(tf.keras.layers.LSTM(64))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(64, activation='relu', kernel_regularizer = regularizers.l2(0.001)))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
    model.summary()
    return model